In [1]:
import pandas as pd
import xlrd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
main_path = './setTraining/select_noOhe/'
train = np.load(main_path+'train.npy')
test = np.load(main_path+'test.npy')
y_train = np.load(main_path+'y_train.npy')
y_test = np.load(main_path+'y_test.npy')
print('shapes: train = {}, y_train = {}, test = {}, y_teste = {}'.format(train.shape,y_train.shape,test.shape,y_test.shape))

shapes: train = (17389, 12), y_train = (17389, 1), test = (8566, 12), y_teste = (8566, 1)


In [3]:
# standardscaler x data
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_trainScaled = sc_x.fit_transform(train)
X_testScaled = sc_x.fit_transform(test)

## Combine Models

### Bagging Algorithms
Building multiple models from different subsamples of the training dataset
### Boosting
Building multiple models each of which learns to fix the prediction errors
### Voting
Building multiple models and simple statistics are used to combine predictions

In [4]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [5]:
y = np.ravel(y_train)
print(y.shape)

(17389,)


In [6]:
num_folds = 10
kfold = KFold(n_splits=10,random_state=7,shuffle=True)

In [8]:
#bagging
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
seed = 7 #
dt = DecisionTreeClassifier()
num_trees = 100 #total de 100 trees created
model = BaggingClassifier(base_estimator = dt, n_estimators = num_trees, random_state = seed)
results = cross_val_score(model,X_trainScaled,y, cv=kfold)
print(results.mean())

0.9497963526781193


In [9]:
#for comparison LR = 0.5290700844565642

In [10]:
#random forest
from sklearn.ensemble import RandomForestClassifier
max_features = 3
model = RandomForestClassifier(n_estimators = num_trees, max_features=max_features)
results = cross_val_score(model,X_trainScaled,y, cv=kfold)
print(results.mean())

0.9600900547978384


In [11]:
#extra trees classification
from sklearn.ensemble import ExtraTreesClassifier
max_features = 7
model = ExtraTreesClassifier(n_estimators = num_trees, max_features= max_features)
results = cross_val_score(model,X_trainScaled,y, cv=kfold)
print(results.mean())

0.9718790344833975


## Adaboost
weighting instances in the dataset by how easy or difficult they are to classify

In [12]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators = num_trees, random_state = seed)
results = cross_val_score(model,X_trainScaled,y, cv=kfold)
print(results.mean())

0.5353970808455053


## Stochastic Gradient

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators = num_trees, random_state=seed)
results = cross_val_score(model,X_trainScaled,y, cv=kfold)
print(results.mean())

0.792800744578283


## Voting Ensemble
combine 2 or more stand-alone models, then b used to wrap your models and average predictions

In [16]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic',model1))
model2 = DecisionTreeClassifier()
estimators.append(('dt',model2))
model3  = SVC()
estimators.append(('svm',model3))
ensemble = VotingClassifier(estimators)
results = cross_val_score(ensemble,X_trainScaled,y, cv=kfold)
print(results.mean())

c:\users\tcnew\onedrive\documentos\doutorado\07_udemy\soft_sensors\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\tcnew\onedrive\documentos\doutorado\07_udemy\soft_sensors\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver

0.7551325742411119


## Grid Search tuning
evaluates different alpha values for Ridge Regression algorithm

In [17]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
param_grid = dict(alpha = alphas)
model = Ridge()
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid.fit(X_trainScaled,y)
print(grid.best_score_)
print(grid.best_estimator_.alpha)

0.5368629672161143
0.01


In [18]:
# Random search - sample algorithm parameters for a random distribution (i.e. uniform) for a fixed number of iterations
# parameters - Ridge Regression, alpha values between 0 and 1, total iterations = 100

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
param_grid = {'alpha': uniform()}
model = Ridge()
rsearch = RandomizedSearchCV(estimator=model, param_distributions = param_grid, n_iter=100, random_state=7)
rsearch.fit(X_trainScaled,y)
print(rsearch.best_score_)
print(rsearch.best_estimator_.alpha)

0.5368638325758534
0.024899227550348013


In [21]:
# for comparison MLP Classifier = 0.4451319168806911